## Important Stuff

Link to data: https://docs.google.com/spreadsheets/d/1vBdD5fDEb8e4J3cmIwoqyLTgelAbumhq2q19hsziX1I/edit?gid=0#gid=0

In [1]:
import pandas as pd
import googlemaps
import folium
import os

In [2]:
df = pd.read_csv('test_data.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Sellers                         20 non-null     object 
 1   Buyers                          20 non-null     object 
 2   Sale Price                      20 non-null     int64  
 3   BBL                             20 non-null     int64  
 4   Building BBL                    18 non-null     float64
 5   Use Code Description            20 non-null     object 
 6   Property Sq. Ft                 10 non-null     float64
 7   Recorded Date of Previous Sale  9 non-null      object 
 8   Doc Date of Previous Sale       9 non-null      object 
 9   Previous Owner Name             6 non-null      object 
 10  Previous Sale Price             9 non-null      float64
 11  Physical Address                20 non-null     object 
 12  Municipality                    20 non

In [7]:
df.head(1)

,Sellers,Buyers,Sale Price,BBL,Building BBL,Use Code Description,Property Sq. Ft,Recorded Date of Previous Sale,Doc Date of Previous Sale,Previous Owner Name,Previous Sale Price,Physical Address,Municipality,County,Neighborhood,Geo-Coordinates,PropAppraiserURL
0,"FORTUNA, FIONA","MC CROSSIN, SEAN",280000,1013360010,1.013360e+09,Residential Co-op,NaN,2007-02-14 00:00:00.000000,2007-02-07 00:00:00.000000,NaN,190000.0,"321 EAST 43RD STREET Unit 511, NEW YORK, NY",New York City,New York County,Turtle Bay,"(-73.97049890662383,40.75023918271339)",https://zola.planning.nyc.gov/l/lot/1/133/60010


In [13]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# --- 1) Parse Geo-Coordinates into lon/lat ---
# Your Geo-Coordinates look like: "(-73.97...,40.75...)"
coords = (
    df["Geo-Coordinates"]
      .astype(str)
      .str.strip("()")
      .str.split(",", expand=True)
)

df["lon"] = pd.to_numeric(coords[0], errors="coerce")
df["lat"] = pd.to_numeric(coords[1], errors="coerce")

# Drop rows missing coords
df_map = df.dropna(subset=["lat", "lon"]).copy()

# --- 2) Choose a starting center (mean of points) ---
center_lat = df_map["lat"].mean()
center_lon = df_map["lon"].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles="CartoDB positron")

# --- 3) Add markers (clustered) with a nice popup ---
cluster = MarkerCluster(name="Sales").add_to(m)

for _, r in df_map.iterrows():
    sale_price = r.get("Sale Price")
    address = r.get("Physical Address", "")
    sellers = r.get("Sellers", "")
    buyers = r.get("Buyers", "")
    neighborhood = r.get("Neighborhood", "")
    url = r.get("PropAppraiserURL", "")

    # Popup HTML (keep it compact but useful)
    popup_html = f"""
    <div style="font-size: 13px; line-height: 1.25;">
      <b>{address}</b><br>
      <b>Sale:</b> ${sale_price:,.0f}<br>
      <b>Sellers:</b> {sellers}<br>
      <b>Buyers:</b> {buyers}<br>
      <b>Neighborhood:</b> {neighborhood}<br>
      <a href="{url}" target="_blank" rel="noopener noreferrer">Open Zola / Lot</a>
    </div>
    """

    folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=6,
        popup=folium.Popup(popup_html, max_width=350),
        tooltip=f"{address} — ${sale_price:,.0f}",
        fill=True
    ).add_to(cluster)

folium.LayerControl().add_to(m)

# --- 4) Save / display ---
m.save("index.html")
m


In [11]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/test_map
